# Redes Neurais Artificiais
 **Nome: João Pedro Miranda Marques**
 
 **Matrícula: 2017050495** 

## Exercicio 4 - Perceptron Simples

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rd

### 1 - Treinamento e vizualização

In [4]:

# Estimate Perceptron weights using stochastic gradient descent
def train_perceptron(xin, yd, eta, tol, maxepocas, par):
# xin : matriz Nxn com os dados de entrada
# yd: r ́otulos de saída (0 ou 1)
# eta : passo de treinamento
# tol : tolerˆancia de erro
# maxepocas: nu ́mero ma ́ximo de itera ̧co ̃es par : parˆametro de entrada .
# # par=0 ==> xin tem dimens ̃ao n+1 e j ́a inclui
# # entrada correspondente ao termo
# # de polarizac ̧ ̃ao.
# # par=1 ==> xin tem dimens ̃ao n e na ̃o inclui

    dimxin = xin.shape  # Dimensões do conjunto de dados.
    N = dimxin[0]       # Numero de amostras.
    n = dimxin[1]       # Dimensao de entrada.

    # Adiciona ou não um termo de polarização ao vetor de treinamento w.
    if(par == 1):
        wt = pd.DataFrame(rd.uniform(n+1) - 0.5).to_numpy() 
        xin = pd.concat(xin, axis=-1) # xin<−cbind(−1,xin)
    else:
        wt = pd.DataFrame(rd.uniform(n) - 0.5).to_numpy() 

    nepocas = 0 # Contador de epocas
    eepoca = tol + 1 # Acumulador de erro de epocas
    evec = [1][maxepocas] # Vetor de erros

    # Laço principal de treinamento
    while((nepocas < maxepocas) & (eepca > tol)):
        ei2 = 0
        #Sequencia aleatória de treinamento
        xseq = rd.sample(N)
        for i in N:
            # Amostra dado da sequencia aleatória
            irand = xseq[i]
            # Calcula saída do Perceptron
            yhati = 1* (np.dot(xin[irand ,], wt ) >= 0)
            ei = yd[irand] - yhati
            dw = eta * ei * xin[irand,]
            # Ajusta vetor de pesos
            wt = wt + dw
            # Acumula erro por época
            ei2 = ei2+ei*ei

        # Incrementa número de épocas
        nepocas = nepocas + 1
        evec[nepocas] = ei2/N
        # Armazena erro por época
        eepoca = evec[nepocas]

    # Retorna vetores de pesos e de erros
    retlist = [wt, evec[1:nepocas]]

    return retlist
    